In [1]:
#Loading the dataset
import pandas as pd 
df= pd.read_csv("movie_data.csv")
df.head(10)

,review,sentiment
0,This movie is just crap. Even though the direc...,0
1,Another detailed work on the subject by Dr Dwi...,1
2,THE CAT O'NINE TAILS (Il Gatto a Nove Code) <b...,0
3,"Like with any movie genre, there are good gang...",0
4,I watched it with my mom and we were like...<b...,0
5,This movie is probably one of 3 worst movies m...,0
6,"this movie is quite bad, aggressive, not playe...",0
7,And a perfect film to watch during the holiday...,1
8,"I like Noel Coward, the wit. I like Noel Cowar...",0
9,"""The Days"" is a typical family drama with a li...",1


In [2]:
df['review'][0]

'This movie is just crap. Even though the directors claim to be part of that oi-culture, it\'s still a very, very bad directorial debut. The topic itself is very interesting and I accept the bad acting due to the fact, that they are all amateurs and never acted before, but the worst thing about this film are the dialogs and very unexperienced and naive directing. There\'s no timing at all in that movie. I felt like the directors were so exited to do that movie (it\'s their first feature), that they actually never really asked themselves, what story they wanna tell. I met Ben (one of the directors) on several occasions and he\'s a nice and thoughtful guy, but that doesn\'t make him a director. I think, that "American History X" is full of clichÃ©s, but somehow manages to transport a story. "Oi!Warning" is full of clichÃ©s, doesn\'t tell anything new or provocative and (-that\'s the sad thing about this movie) it\'s far from any Oi!-Reality.<br /><br />If you wanna see weird but great Ge

# Transforming documents into feature vectors

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs=np.array(['The sun is shining',
             'The weather is sweet',
            'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
                      

In [4]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [5]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


# Word relevancy using term frequency-inverse document frequency



In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)

In [7]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [8]:
df.loc[0, 'review'][-50:]

't of ten, because of the topic and the photography'

In [9]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [10]:
preprocessor(df.loc[0, 'review'][-50:])

't of ten because of the topic and the photography'

# Tokenizing Documents

In [11]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

In [12]:
def tokenizer(text):
    return text.split()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words("english")

# Transform Text Data into TF-IDF Vectors

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None, tokenizer= tokenizer_porter, use_idf=True, norm="l2", smooth_idf=True)
y= df.sentiment.values
X = tfidf.fit_transform(df.review)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, test_size=0.5, shuffle=False)

In [18]:

from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5,
                        scoring='accuracy',
                        random_state=0,
                        n_jobs=-1,
                        verbose=3,
                        max_iter=300).fit(X_train,y_train)
clf.score(X_test, y_test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  6.5min remaining:  9.7min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  6.5min finished


0.8896938162897738